In [0]:
FIELD = 'ursaminor'
CONFIG_FILES = [
    '/home/dobos/project/ga_targeting/configs/netflow/SSP/dSph/_common.py',
    f'/home/dobos/project/ga_targeting/configs/netflow/SSP/dSph/{FIELD}.py'
]

TIME_FROM = '2025-04-01 12:00:00'       # HST
TIME_TO = '2025-04-02 12:00:00'

# Calculate the rotator angle as a function of time

In [0]:
from datetime import datetime, timedelta
import dateutil.parser
import numpy as np
import matplotlib.pyplot as plt

from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astroplan import Observer
import astropy.units as u

from pfs.utils.coordinates import Subaru_POPT2_PFS

from pfs.ga.targeting.config.netflow import NetflowConfig
from pfs.ga.targeting.targets.dsph import GALAXIES

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
obs = Observer.at_site("Subaru", timezone="US/Hawaii")
hawaii_tz = TimeDelta(-10 * u.hr)

In [0]:
# Load config
field = GALAXIES[FIELD]
config = field.get_netflow_config()
config.load(CONFIG_FILES, ignore_collisions=True)

In [0]:
config.pointings

In [0]:
time_from = np.min([ p.obs_time for p in config.pointings ])
time_to = np.max([ p.obs_time for p in config.pointings ])

time_from, time_to

In [0]:
time_from = Time(dateutil.parser.parse(TIME_FROM)) - hawaii_tz
time_to = Time(dateutil.parser.parse(TIME_TO)) - hawaii_tz

time_from, time_to

In [0]:
date_from = np.floor(time_from.jd - 0.5)
date_to = np.ceil(time_to.jd + 0.5)

dates = Time(np.linspace(date_from, date_to + 1, int(date_to - date_from + 2)), format='jd')
dates.to_datetime()

In [0]:
sunrise = obs.sun_rise_time(dates, which='previous') + hawaii_tz
sunset = obs.sun_set_time(dates, which='next') + hawaii_tz

# HST
sunrise.to_datetime(), sunset.to_datetime()

In [0]:
def radecpa_to_altazinr(pos, pa, obs_time):
    altaz = pos.transform_to(AltAz(obstime=obs_time, location=Subaru_POPT2_PFS.Lsbr))
    az = altaz.az.deg
    alt = altaz.alt.deg

    # Instrument rotator angle
    subaru = Subaru_POPT2_PFS.Subaru()
    paa = subaru.radec2inr(pos.ra, pos.dec, obs_time)
    inr = (paa + pa + 180) % 360. - 180.
    # inr = paa + pa

    return alt, az, inr

In [0]:
obs_time = Time(np.linspace(date_from, date_to, 300), format='jd')
obs_time[0], obs_time[-1]

In [0]:
obs_time[0], obs_time[-1]

In [0]:
f, ax = plt.subplots(1, 1, figsize=(7, 5), dpi=240)
ax2 = ax.twinx()

xx = (obs_time + hawaii_tz).to_datetime()

# Sun up
for t1, t2 in zip(sunset[:-2].to_datetime(), sunrise[2:].to_datetime()):
    ax.axvspan(t1, t2, color='black', alpha=1)


for p in config.pointings:
    p = p.get_pointing()
    alt, az, inr = radecpa_to_altazinr(p.pos, p.posang, obs_time=(obs_time))
    # alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))

    # Instrument rotator angle
    ax2.plot(xx, inr, c='y')

# Target elevation - plot once
p = field.get_center_pointing(include_motion=False)
alt, az, inr = radecpa_to_altazinr(p.pos, 0, obs_time=(obs_time))
ax.plot(xx, alt)

ax.set_xlim((time_from + hawaii_tz).to_datetime(), (time_to + hawaii_tz).to_datetime())

ax.set_yticks(np.linspace(0, 90, 7))
ax.set_ylim(0, 90)

ax.tick_params(axis='x', labelrotation=90)

ax.set_xlabel('time [HST]')
ax.set_ylabel('elevation')
ax.set_title(FIELD)

ax.grid(True)

ax2.set_yticks(np.linspace(-180, 180, 7))
ax2.set_ylim(-180, 180)

ax2.set_ylabel('rotator angle')

In [0]:
from pfs.utils.coordinates.DistortionCoefficients import radec_to_subaru

In [0]:
az, el, inr = radec_to_subaru(227.2833, +67.2225, -20, datetime(2025, 3, 25, 0, 0, 0), 2016.0, 0, 0, 1e-7)
az, el, inr

In [0]:
from pfs.ga.targeting.util.args import *
normalize_angle(-20)

In [0]:
alt, az, inr = radecpa_to_altazinr(normalize_pos(227.2833, +67.2225), -20, datetime(2025, 3, 25, 0, 0, 0))
az, alt, inr